# Pure Python Walkthrough


Setup the environment

In [1]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")


# configure logfire
logfire.configure(token=logfire_token)

# 2. Instrumenteer LangChain (automatisch loggen van chains, LLM calls, tools, etc.)
logfire.instrument()

<function logfire._internal.instrument.instrument.<locals>.decorator(func: 'Callable[P, R]') -> 'Callable[P, R]'>

Logfire project URL: ]8;id=436660;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\


1 Definieer een basis agent

In [2]:
dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

import json
from anthropic import Anthropic
from typing import Dict, List, Optional, Callable, Any


class LLMAgent:
    def __init__(self, name: str, system_prompt: str = ""):
        """
        Initialiseer de LLM-agent.

        Args:
            name: Naam van de agent
            system_prompt: Systeeminstructies voor de agent
        """
        self.client = Anthropic(api_key=api_key)
        self.model = model
        self.name = name
        self.system_prompt = system_prompt

    def execute_tool(self, tool_name: str, **kwargs) -> str:
        """
        Voer een tool uit met de gegeven parameters.
        """
        if tool_name in self.tools:
            try:
                result = self.tools[tool_name]["function"](**kwargs)
                return f"Tool '{tool_name}' executed successfully. Result: {result}"
            except Exception as e:
                return f"Error executing tool '{tool_name}': {str(e)}"
        return f"Tool '{tool_name}' not found."

    @logfire.instrument()
    def run(self, user_input: str) -> str:
        """
        Verwerk gebruikersinvoer en genereer een reactie.
        """
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            system=self.system_prompt,
            messages=[{"role": "user", "content": user_input}],
        )

        # Extract the response text
        response_text = (
            response.content[0].text if hasattr(response, "content") else response
        )
        # Log the response to Logfire
        logfire.info(
            "Agent response generated",
            agent_name=self.name,
            user_input=user_input,
            response=response_text,
            model=self.model,
            response_length=len(response_text),
        )

        return response


basic_agent = LLMAgent(
    name="basic_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

Stel een vraag aan de agent

In [3]:
# Run the agent
result = basic_agent.run("geef een defintie van een agent")


print(result.content[0].text)

17:16:22.531 Calling __main__.LLMAgent.run
17:16:25.575   Agent response generated
Een agent is een computerprogramma of systeem dat autonoom handelt om een bepaald doel te bereiken. De belangrijkste kenmerken van een agent zijn:

1. Autonomie: Een agent kan zelfstandig beslissingen nemen en acties ondernemen zonder constante menselijke sturing.

2. Waarneming: Een agent neemt informatie waar uit zijn omgeving, zoals input van sensoren of data, en gebruikt deze informatie om beslissingen te nemen.

3. Doelgerichtheid: Een agent heeft een doel of set van doelen die hij probeert te bereiken door middel van zijn acties.

4. Flexibiliteit: Een agent kan zich aanpassen aan veranderende omstandigheden en nieuwe situaties om zijn doelen te bereiken.

5. Intelligentie: Een agent gebruikt cognitieve vaardigheden zoals redeneren, leren en probleemoplossing om efficiënt te handelen.

Agenten kunnen in veel verschillende domeinen worden toegepast, zoals robotica, games, verkeersmanagement, e-comme

Definieer tools voor de agent

In [4]:
# Anthropic Tool Definitions (voor Pure Python)
ANTHROPIC_TOOLS = [
    {
        "name": "calculate_btw",
        "description": "Bereken BTW (belasting toegevoegde waarde) over een bedrag. Standaard BTW tarief is 21%.",
        "input_schema": {
            "type": "object",
            "properties": {
                "bedrag": {"type": "number", "description": "Het bedrag exclusief BTW"},
                "btw_percentage": {
                    "type": "number",
                    "description": "BTW percentage (standaard 21%)",
                    "default": 21.0,
                },
            },
            "required": ["bedrag"],
        },
    },
    {
        "name": "calculate_discount",
        "description": "Bereken korting over een bedrag.",
        "input_schema": {
            "type": "object",
            "properties": {
                "bedrag": {
                    "type": "number",
                    "description": "Het oorspronkelijke bedrag",
                },
                "korting_percentage": {
                    "type": "number",
                    "description": "Korting percentage",
                },
            },
            "required": ["bedrag", "korting_percentage"],
        },
    },
    {
        "name": "get_inwoners_gemeente",
        "description": "Haal inwonersaantallen op van Nederlandse gemeenten via CBS API. Kan specifieke gemeente opzoeken of top 10 grootste gemeenten tonen.",
        "input_schema": {
            "type": "object",
            "properties": {
                "gemeente_naam": {
                    "type": "string",
                    "description": "Naam van de gemeente (optioneel). Laat leeg voor top 10 grootste gemeenten.",
                },
                "jaar": {
                    "type": "string",
                    "description": "Jaar voor de data (standaard 2023)",
                    "default": "2023",
                },
            },
            "required": [],
        },
    },
]

# Tool function mapping
TOOL_FUNCTIONS = {
    "calculate_btw": tf.calculate_btw,
    "calculate_discount": tf.calculate_discount,
    "get_inwoners_gemeente": tf.get_inwoners_gemeente,
}

In [5]:
class ToolAgent(LLMAgent):
    def __init__(self, name: str, system_prompt: str = ""):
        super().__init__(name, system_prompt)
        self.tools = ANTHROPIC_TOOLS

    @logfire.instrument()
    def run(self, user_input: str) -> str:

        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            tools=self.tools,
            messages=[{"role": "user", "content": user_input}],
        )
        # Extract the response text
        response_text = (
            response.content[0].text if hasattr(response, "content") else response
        )
        # Log the response to Logfire
        logfire.info(
            "Agent response generated",
            agent_name=self.name,
            user_input=user_input,
            response=response_text,
            model=self.model,
            response_length=len(response_text),
        )

        # Check if Claude wants to use tools
        if response.stop_reason == "tool_use":
            tool_results = []

            for content in response.content:
                if content.type == "tool_use":
                    tool_name = content.name
                    tool_input = content.input
                    tool_id = content.id

                    print(
                        f"🔧 Claude gebruikt tool: {tool_name} met input: {tool_input}"
                    )

                    # Execute the tool
                    if tool_name in TOOL_FUNCTIONS:
                        result = TOOL_FUNCTIONS[tool_name](**tool_input)
                        tool_results.append(
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_id,
                                "content": json.dumps(result),
                            }
                        )
                    else:
                        print(f"Tool {tool_name} not found")

            # Get final response with tool results
            final_response = self.client.messages.create(
                model=self.model,
                max_tokens=1000,
                tools=self.tools,
                messages=[
                    {"role": "user", "content": user_input},
                    {"role": "assistant", "content": response.content},
                    {"role": "user", "content": tool_results},
                ],
            )
            # Log the final response to Logfire
            logfire.info(
                "Agent response generated",
                agent_name=self.name,
                user_input=user_input,
                response=final_response,
            )
            return final_response
        else:
            return response


tool_agent = ToolAgent(
    name="tool_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

Test een aantal tool calls

In [6]:
result2 = tool_agent.run(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)

print(result2.content[0].text)

17:16:50.159 Calling __main__.ToolAgent.run
17:16:51.592   Agent response generated
🔧 Claude gebruikt tool: calculate_btw met input: {'bedrag': 100}
17:16:52.974   Agent response generated
Het bedrag exclusief BTW is €100. Met het standaard BTW-tarief van 21% komt daar €21 BTW bij, zodat het totale bedrag inclusief BTW €121 is.


In [7]:
result = tool_agent.run(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2025. Laat ook het aantal inwoners van Groningen zien."
)

print(result.content[0].text)

17:16:54.232 Calling __main__.ToolAgent.run
17:16:55.437   Agent response generated
🔧 Claude gebruikt tool: get_inwoners_gemeente met input: {'jaar': '2025'}
17:16:58.080   Agent response generated
De top-7 gemeenten in Nederland met het grootste aantal inwoners in 2025 zijn:
1. Amsterdam (935.793)
2. Rotterdam (672.330) 
3. 's-Gravenhage (568.419)
4. Utrecht (376.435)
5. Eindhoven (249.054)
6. Groningen (244.807)
7. Tilburg (230.359)

Het aantal inwoners van Groningen in 2025 wordt geschat op 244.807.


In [8]:
result = tool_agent.run(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan.",
)
print(result.content[0].text)

17:17:07.506 Calling __main__.ToolAgent.run
17:17:09.692   Agent response generated
🔧 Claude gebruikt tool: get_inwoners_gemeente met input: {'gemeente_naam': 'Eindhoven', 'jaar': '2023'}
17:17:12.042   Agent response generated
Volgens de data van het CBS had Eindhoven in 2023 243.730 inwoners.

Als alle inwoners in 2025 15 euro betalen, dan zou dat het volgende opleveren:


Voeg geheugen toe aan de agent

In [9]:
class MemAgent(ToolAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.memory = []

    def add_memory(self, role, memory):
        self.memory.append({"role": role, "content": memory})

    def get_memory(self):
        return self.memory

    def run(self, user_input):
        self.memory.append(user_input)
        input = ""
        for message in self.memory:
            input += f"role': 'user': 'content': {message}\n"

        return super().run(input)


mem_agent = MemAgent(
    name="mem_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

In [10]:
result = mem_agent.run(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)

print(result.content[0].text)

17:17:19.478 Calling __main__.ToolAgent.run
17:17:23.415   Agent response generated
🔧 Claude gebruikt tool: calculate_btw met input: {'bedrag': 100}
17:17:25.645   Agent response generated
Het product kost 100 euro exclusief BTW. Bij een BTW percentage van 21% is de BTW 21 euro. Het totaalbedrag inclusief BTW is dan 121 euro.


In [12]:
result = mem_agent.run(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor."
)

print(result.content[0].text)

17:18:07.066 Calling __main__.ToolAgent.run
17:18:08.520   Agent response generated
🔧 Claude gebruikt tool: calculate_btw met input: {'bedrag': 100}
17:18:09.762   Agent response generated
Het BTW bedrag voor het product X van 100 euro exclusief BTW is 21 euro.
